In [1]:
## Bibliotecas
import pandas as pd
import bokeh.plotting as bp
import sys
sys.path.append(r'C:\Users\giovanni.5683\GITHUB\controlling_postgreSQL')  # Adiciona o diretório onde está conn.py
from conn import get_connection
engine = get_connection()
cursor = engine.cursor()

In [7]:
## Conexão e leitura da view
Pendente = """SELECT * FROM byd.bonus_view"""
df_pendente = pd.read_sql_query(Pendente, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_47556\3687575432.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pendente = pd.read_sql_query(Pendente, engine)


In [10]:
## Filtragem das colunas
df_pendente = df_pendente[['idnfsexterno','des_modelo', 'apontamento', 'competencia', 'bonus_utilizado', 'valor_bonus', 'trade', 'bonus_dpto', 'trade_mkt_dpto']]
df_pendente

,idnfsexterno,des_modelo,apontamento,competencia,bonus_utilizado,valor_bonus,trade,bonus_dpto,trade_mkt_dpto
0,A25519901V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Outubro,Equalização,14221.4,None,14221.4,None
1,A25252731V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Outubro,Equalização,14221.4,None,14221.4,None
2,A251250881V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Setembro,Equalização,9221.4,None,9221.4,None
3,A2583641V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Certo!,Outubro,Equalização,14221.4,None,14221.4,None
4,A251214891V210,DOLPHIN 70KW Motor 405km - Afterglow Pink - An...,Revisar Divergência!,Julho,Varejo,17000.0,None,None,None
...,...,...,...,...,...,...,...,...,...
5143,A25720988653888VD,BYD KING GL DM-I,,Dezembro,Venda Direta,None,None,None,None
5144,A2571016353909VD,DOLPHIN MINI GL BRANCO INT AZ ESC,,Dezembro,Venda Direta,None,None,None,None
5145,A251208468516728VD,SONG PRO DMI GL,Certo!,Dezembro,Venda Direta,0.00,None,None,None
5146,A2519164517168VD,I/BYD DOLPHIN MINI,Certo!,Dezembro,Venda Direta,0.00,None,None,None


In [9]:
## Filtragem dos dados 'Revisar Divergência!' para os meses de Agosto a Dezembro
df_pv = df_pendente[
    (df_pendente['apontamento'] == 'Revisar Divergência!') &
    (df_pendente['competencia'].isin(['Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']))
]
df_pv

,idnfsexterno,des_modelo,apontamento,competencia,bonus_utilizado,valor_bonus,trade,bonus_dpto,trade_mkt_dpto


In [5]:
## Atualização dos dados na tabela byd.controladoria
for index, row in df_pv.iterrows():
    idnfsexterno = row['idnfsexterno']
    valor_bonus = row['valor_bonus']
    trade = row['trade']
    
    update_query = """
    UPDATE byd.controladoria 
    SET bonus_dpto = %s, trade_mkt_dpto = %s 
    WHERE idnfsexterno = %s
    """
    cursor.execute(update_query, (valor_bonus, trade, idnfsexterno))

engine.commit()